In [9]:
import psycopg2, configparser
import pandas as pd

#Get the API-key
config_obj = configparser.ConfigParser()
config_obj.read("config.ini")

['config.ini']

In [5]:
database_logins = config_obj["POSTGRELOGINS"]

conn = psycopg2.connect(
    database="beers",
    user=database_logins['USERNAME'],
    password=database_logins['PASSWORD'],
    host=database_logins['HOST'],
    port=database_logins['PORT']
)

conn.autocommit = True

In [6]:
cursor = conn.cursor()

In [108]:
create_tables = open("datasets/create_tables.sql", "r")
create_statements = create_tables.read()
create_tables.close()
cursor.execute(create_statements)

In [109]:
beer_dataset = pd.read_csv('datasets/beer_dataset.csv')

In [110]:
for index, row in beer_dataset.iterrows():
    try:
        sql = '''
        INSERT into inventory (Name, Brewery, Type, Alcohol, Stock, Strong, Country)
        VALUES ('%s', '%s', '%s', '%s', %s, '%s', '%s')
        ''' % (row['Name'].replace("'","\'\'"), row['Brewery'].replace("'","\'\'"), row['Type'], row['Alcohol'], row['Stock'], \
               row['Strong'], row['Country'])
        cursor.execute(sql)
    except Exception as e:
        print(sql)
        print(e)
        break
    


In [123]:
cursor.execute("""
SELECT *
FROM inventory
WHERE Stock > 50
    """)

In [124]:
result = pd.DataFrame(cursor.fetchall())

In [125]:
result.head(20)

,0,1,2,3,4,5,6,7
0,3,Abbaye de Bonne-Esperance,Lefebvre,amber,8.00%,93,strong,Belgium
1,5,Abbaye de Cambron,Brasserie de Silly,amber,6.50%,72,normal,Belgium
2,7,Abbaye de la Moinette brune,Dupont,brown,8.50%,59,strong,Belgium
3,8,Abbaye de la Moinette brune,Dupont,brown,7.50%,51,strong,Belgium
4,9,Abbaye de la Moinette-Special Nouvel An,Dupont,amber,8.50%,100,strong,Belgium
5,13,Abbaye de Leffe vieille cuvée,St. Guibert,brown,7.80%,63,strong,Belgium
6,15,Abbaye des Rocs Special,Eloir-Bertiau,unknown,9.50%,70,strong,Belgium
7,16,Abbaye de Leffe brown,St. Guibert,amber,6.30%,98,normal,Belgium
8,17,Abdijbier Affligem brown,De Smedt,brown,6.90%,77,normal,Belgium
9,19,Abdijbier Affligem trippel,De Smedt,amber,9.60%,79,strong,Belgium


In [106]:
cursor.execute('DROP TABLE inventory')